In [8]:
import networkx as nx
import pandas as pd
import numpy as np
from haversine import haversine, Unit

import sys 
sys.path.append('..')
from utils import dataloader

In [7]:
dist_matrix = pd.read_csv('../data/distance_matrix.csv')
dist_matrix.head()
df = dataloader.load_data()

In [ ]:
def get_graph(data):
    G = nx.Graph()

    for cluster in data['Cluster'].unique():
        G.add_node(cluster)
        G.nodes[cluster]['ID'] = dist_matrix.columns.get_loc('WEBSTER')
        G.nodes[cluster]['lat'] = data[data['ID'] == cluster]['Latitude'].iloc[0]
        G.nodes[cluster]['long'] = data[data['ID'] == cluster]['Longitude'].iloc[0]
        G.nodes[cluster]['pos'] = (G.nodes[cluster]['long'], G.nodes[cluster]['lat'])


    for node_x in G.nodes:
        for node_y in G.nodes:
            dist = haversine(
                    (df[df['Cluster'] == cluster]['Latitude'].mean(), df[df['Cluster'] == cluster]['Longitude'].mean()), 
                    (df[df['Cluster'] == cluster]['Latitude'].mean(), df[df['Cluster'] == cluster]['Longitude'].mean()),
                    unit=Unit.KILOMETERS)
            if (dist > 2.5):
                continue
            G.add_edge(node_x, node_y)
            G[node_x][node_y]['weight'] = np.exp(-dist)


    adj = nx.adjacency_matrix(G)
    return G, adj.todense()
    #return G, normalize_adj(adj).todense()

In [5]:
pd.read_csv('../data/charging_session_count_30.csv')

,Period,BRYANT,CAMBRIDGE,HAMILTON,HIGH,MPL,RINCONADA,TED,WEBSTER
0,2011-07-29 20:00:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2011-07-29 20:30:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2011-07-29 21:00:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2011-07-29 21:30:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2011-07-29 22:00:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
156657,2020-07-05 12:30:00,0.0,9.0,0.0,1.0,0.0,0.0,0.0,1.0
156658,2020-07-05 13:00:00,0.0,6.0,0.0,1.0,1.0,1.0,0.0,1.0
156659,2020-07-05 13:30:00,0.0,4.0,0.0,1.0,1.0,1.0,1.0,2.0
156660,2020-07-05 14:00:00,0.0,3.0,0.0,1.0,1.0,1.0,1.0,2.0


In [16]:
dist_matrix.columns.get_loc('WEBSTER')

5